In [1]:
import tensorflow as tf
import pickle
import numpy as np
from custom_models import InferenceDropout

In [2]:
with open('x_train_values.pickle', 'rb') as file:
    x_train_values = pickle.load(file)

with open('y_train_values.pickle', 'rb') as file:
    y_train_values = pickle.load(file)

with open('x_val_values.pickle', 'rb') as file:
    x_val_values = pickle.load(file)

with open('y_val_values.pickle', 'rb') as file:
    y_val_values = pickle.load(file)
print(x_val_values[0][:10], y_val_values[0])

[array([  0.38072145,   0.83115378, -22.68116   ,  22.059294  ]), array([  0.38514185,   1.00016554, -47.037086  ,  36.429188  ]), array([  0.38568805,   0.83115378, -10.826701  ,   6.088397  ]), array([ 0.38617675,  1.00016554, -4.541746  , 21.268929  ]), array([ 0.38672025,  1.00016554,  7.537838  , 32.004265  ]), array([0.39164805, 0.42261215, 2.327167  , 4.492903  ]), array([ 0.39471245,  0.83115378,  3.470206  , 18.749502  ]), array([ 0.39515445,  0.19152458, -0.729853  ,  2.88595   ]), array([0.3956181 , 0.64161563, 6.184633  , 4.776905  ]), array([ 0.3982013 ,  0.42261215, -3.335007  ,  7.256248  ])] 0.6084354999999996


In [3]:
tf.random.set_seed(42)


model = tf.keras.Sequential([
    tf.keras.layers.Input((352, 4)),
    tf.keras.layers.Masking(mask_value=0.0),
    tf.keras.layers.GRU(50, return_sequences=True, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(50,activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 352, 4)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 352, 50)        │         8,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 352, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 50)             │        15,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,751 (92.78 KB)

 Trainable params: 23,751 (92.78 KB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='mean_squared_error')

In [5]:
model.fit(np.array(x_train_values), np.array(y_train_values),
          epochs=50', validation_data=(np.array(x_val_values), np.array(y_val_values)))

Epoch 1/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 77s 216ms/step - loss: 0.0166 - val_loss: 0.0484
Epoch 2/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 77s 220ms/step - loss: 0.0059 - val_loss: 0.0478
Epoch 3/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 77s 222ms/step - loss: 0.0033 - val_loss: 0.0506
Epoch 4/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 78s 224ms/step - loss: 0.0022 - val_loss: 0.0525
Epoch 5/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 81s 231ms/step - loss: 0.0016 - val_loss: 0.0494
Epoch 6/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 77s 222ms/step - loss: 0.0014 - val_loss: 0.0502
Epoch 7/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 74s 213ms/step - loss: 0.0012 - val_loss: 0.0508
Epoch 8/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 74s 214ms/step - loss: 0.0011 - val_loss: 0.0497
Epoch 9/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 75s 216ms/step - loss: 9.5979e-04 - val_loss: 0.0490
Epoch 10/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 73s 209ms/step - loss: 8.9176e-04 - val_loss: 0.0511
Epoch 11/50
348/348 ━━━━━━━━━━━━━━━━━━━━ 64s 184ms/step - loss: 8.2351e-04 - val_loss: 0.0517
Epoch 1

In [6]:
model.save('model.keras')